# Lets first solve the 1-body problem without dissipation
Let 
$$\alpha = -GM$$

In [3]:
#Define alpha as a global variable
G = 1
M = 1
alpha = -G*M 

In [8]:
#Define functions that calculate the energy and the angular momentum of the system
def E(r,vr,vphi):
    '''Takes the radius, radial velocity and angular velocity at a given time and returns the energy.
        r :radius
        vr: radial velocity
        vphi: angular velocity.
        As the problem has axial symmetry, there is no need for the angular position.'''
    T = 0.5 *(vr**2 + (r*vphi)**2)
    U = alpha/r
    return T+U

def L(r,vphi):
    '''Takes the radius and angular velocity at a given time and returns the angular momentum
        r : radius
        vphi: angular velocity'''
    return r**2 * vphi

In [18]:
import numpy as np
import random 

In [13]:
def r(phi,p,e):
    '''Takes the angular position, eccentricity and semi-latus rectum, returns the radial position
    p : semi-latus rectum
    e: eccenticity
    phi:angular position
    '''
    r = p/(1-e*np.cos(phi))
    return r

The orbit equation is 

$$r(\phi) = \frac{p}{1-ecos(\phi)}$$

$$e = \sqrt{1+\frac{2EL^2}{\alpha^2}}$$

$$p = \frac{L^2}{\alpha}$$

In [39]:
#Define the initial conditions of the system
r0 = random.uniform(0,100)
phi0 = 0 #For convention
vr0 = random.uniform(0,10)
vphi0 = random.uniform(0,10)
#Caracterization of the orbit
E = E(r0,vr0,vphi0)
L = L(r0,vphi0)
p = L**2/alpha
e = np.sqrt(1+ 2*E*L**2/alpha**2)
#Simulation conditions
t0 = 0
tf = 100
dt = 0.01 #step size
n = int((tf-t0) / dt)
#Where the data is stored
t = np.linspace(t0, tf, n) # Time information
Q = np.zeros([n,4]) # Information of the state of the system
#Include the initial conditions in Q 
Q[0] = np.array([r0,phi0,vr0,vphi0])

For the main loop we will use that the state of the system at $t+\Delta t$ is:
    $$r(\phi+ \Delta \phi ), \phi(t +  \Delta t), \dot r(\phi+ \Delta \phi), \dot \phi(r(\phi+\Delta \phi)).$$
From here that we must obtain each variable in the following order:

1. $$\phi(t +  \Delta t)=\phi + \dot \phi \Delta t$$



2. $$r(\phi+ \Delta \phi ) = \frac{p}{1-ecos(\phi(t+\Delta t))} $$



3. $$\dot \phi(r(\phi+\Delta \phi)) = \frac{L}{r^2(\phi + \Delta \phi)}$$



4. $$\dot r(\phi+ \Delta \phi) = \frac{1}{p}r^2(\phi + \Delta \phi)e\sin(\phi(t+\Delta t))\dot \phi(r(\phi + \Delta \phi))$$

In [46]:
#Define how the above variables are calculated. 
def dphi(vphi,dt):
    '''Takes the instantaneous rate of change and the step size, returns the total change'''
    return vphi * dt

def r(phi,p,e):
    '''Takes the angular position, eccentricity and semi-latus rectum, returns the radial position
    p : semi-latus rectum
    e: eccenticity
    phi:angular position
    '''
    r = p/(1-e*np.cos(phi))
    return r

def vphi(r,L):
    '''takes  the angular momentum  and radius, uses the angular momentum definition
    to obtain the angular velocity at a given r.
    vphi : angular velocity
    L : angular momentum
    r: radius'''
    return L/r**2

def vr(r,phi,vphi,p,e):
    '''Takes the radial and angular position, semi-latus rectum, eccentricity and the 
    angular velocity at a given time. Returns the radial velocity.
    r: radial position
    phi: angular position
    vphi: angular velocity
    p: semi-latus rectum
    e: eccentricity
    '''
    return 1/p * r**2 * e * np.sin(phi) * vphi

In [47]:
#Define the function that updates the state of the system
def updateState(q,dt):
    '''Takes as input an array and a time step. Returns the updated state of the system
    as an array'''
    qUpdated = np.zeros(4)
    #Calculate the new phi
    phiNew = dphi(q[3],dt) + q[1]
    qUpdated[1] = phiNew
    #Calculate the new r
    rNew = r(q[1],p,e)
    qUpdated[0] = rNew
    #New phi dot
    vphiNew = vphi(rNew,L)
    qUpdated[3] = vphiNew
    #New r dot
    vrNew = vr(rNew,phiNew,vphiNew,p,e)
    qUpdated[2] = vrNew
    return qUpdated

In [48]:
#Define how to pass from polar to cartesian 
def polarToCartesian(r,theta):
    '''Takes as input polar coordinates and returns cartesian coordinates'''
    x = np.cos(theta) * r
    y = np.cos(theta) * r
    return x,y 